In [1]:
import numpy as np
import pandas as pd
import spacy
import re

#### Step 1 : Build the data inorder to train the custom NER model. The model accepts training and val data in the below format. (for spacy v3.0 the format is different,  but we need to have the data in below format in first place)

("Flipkart is recognized as leader in market",{"entities": [(0,8, "ORG")]})

###### *Raw data can be converted to above format using the annotators.

#### Step 2 : Downloading the converted data (json) and changing into text

In [6]:
import json

filename = input("Enter your train data filename(json) : ")
print(filename)


with open(filename) as train_data:
    train = json.load(train_data)

TRAIN_DATA = []
for data in train:
    ents = [tuple(entity[:3]) for entity in data['entities']]
    TRAIN_DATA.append((data['content'],{'entities':ents}))


with open('{}'.format(filename.replace('json','txt')),'w') as write:
    write.write(str(TRAIN_DATA))

print("[INFO] Stored the spacy training data and  filename is {}".format(filename.replace('json','txt')))

Enter your train data filename(json) : val100.json
val100.json
[INFO] Stored the spacy training data and  filename is val100.txt


#### Step 3: Cleaning the data (txt) to remove leading and trailing white spaces from entity spans

In [7]:
def trim_entity_spans(data: list) -> list:
    """Removes leading and trailing white spaces from entity spans.

    Args:
        data (list): The data to be cleaned in spaCy JSON format.

    Returns:
        list: The cleaned data.
    """
    invalid_span_tokens = re.compile(r'\s')

    cleaned_data = []
    for text, annotations in data:
        entities = annotations['entities']
        valid_entities = []
        for start, end, label in entities:
            valid_start = start
            valid_end = end
            while valid_start < len(text) and invalid_span_tokens.match(
                    text[valid_start]):
                valid_start += 1
            while valid_end > 1 and invalid_span_tokens.match(
                    text[valid_end - 1]):
                valid_end -= 1
            valid_entities.append([valid_start, valid_end, label])
        cleaned_data.append([text, {'entities': valid_entities}])
        
    return cleaned_data

In [9]:
TRND1 = [('"Dear Candidate, Zomato ke sath judiye aur kamayein Rs. 35000 har mahine, saath hi Rs.2000 se 5000 tak ka joining bonus bhi paayein*. Zomato ke saath food delivery karein aur paayein weekly payment + Insurance benefit + Flexible work timing(Full time/Part time available). OnlineÃ‚Â\xa0register karne ke liye neeche diye link se Zomato app download karein aur register karein. Zomato app link:bit.ly/zomatodeliveryapp,Ã‚Â\xa0Joining process ke baare me aur jaankari ke liye yah video dekhein:https://youtu.be/5SpXmdITXDo', {'entities': [(17, 23, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Sewing Machine Operator in Bangalore with Silver Spark Apparel Limited under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 12000 to 15000 + Benefits. Apply. https://jobalertbot.com/u/20160119/1/14168805', {'entities': [(101, 129, 'Client')]}), ('"Dear Candidate, Zomato ki taraf se Namaskar!! Zomato ke sath judiye aur kamayein Rs. 35000 har mahine ya Rs. 8000 har hafte saath hi Rs.2000 se 5000 tak ka joining bonus bhi paayein*. Zomato ke saath food delivery karein aur paayein weekly payment + Insurance benefit + Flexible work timing(Full time/Part time available). OnlineÃ‚Â\xa0register karne ke liye neeche diye link ko copy karein aur Zomato app download kar register karein. Zomato app link:bit.ly/zomatodeliveryapp,Ã‚Â\xa0Joining process ke baare me aur jaankari ke liye yah video dekhein:https://youtu.be/5SpXmdITXDo', {'entities': [(47, 53, 'Client')]}), ('Urgent: Hiring for Garment Cutter at Fortune Inc in Agra, Uttar Pradesh. Salary 9000 to 10500. Apply here: https://jobalertbot.com/u/22993109/1/14174534', {'entities': [(37, 48, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21065227/1/14168887', {'entities': [(46, 62, 'Client')]}), ('Urgent: Hiring for Driver at Indra Manpower Consultancy in Nellore, Andhra Pradesh. Salary 10000 to 12000. Apply here: https://jobalertbot.com/u/22825864/1/14174490', {'entities': [(29, 55, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Tailor / Sewing machine operator in Tiruppur, with REAL TEXTILES under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 8000 to 10000 + Benefits. Apply. https://jobalertbot.com/u/22972265/1/14174560', {'entities': [(110, 123, 'Client')]}), ('Urgent: Hiring for General Duty Assistant in Portea Medical at Coimbatore, Tamil Nadu. Salary 10000 to 11000. Apply here: https://jobalertbot.com/u/22708645/1/14174432', {'entities': [(45, 59, 'Client')]}), ('Urgent: Hiring for Desktop Engineer/Operation Support Engineer at Denovo Hitech Solutions Pvt Ltd in Mumbai, Maharashtra. Salary 10000 to 15000. Apply here: https://jobalertbot.com/u/22121478/1/14169073', {'entities': [(66, 97, 'Client')]}), ('Urgent: Hiring for General Duty Assistant in Portea Medical at Kolkata, West Bengal. Salary 10000 to 11000. Apply here: https://jobalertbot.com/u/20420031/1/14168822', {'entities': [(45, 59, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21085084/1/14168887', {'entities': [(46, 62, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Delivery Boys in Bangalore withÃ‚Â\xa0 Prince HR services private limited under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 25000 to 35000 + Benefits. Apply. https://jobalertbot.com/u/22177712/1/14169070', {'entities': [(93, 127, 'Client')]}), ('Urgent: Hiring for Car Washer at Indra Manpower Consultancy in Nellore, Andhra Pradesh. Salary 9000 to 10000. Apply here: https://jobalertbot.com/u/22849908/1/14174486', {'entities': [(33, 59, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator(Tailoring) at Stellar Fashion Incorp in Karur, Tamil Nadu. Salary 8000 to 10000. Apply here: https://jobalertbot.com/u/22138862/1/14169074', {'entities': [(57, 79, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21067480/1/14168887', {'entities': [(46, 62, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Helper in Tiruppur, with Zaron Industries under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 6000 to 9000 + Benefits. Apply. https://jobalertbot.com/u/23057838/1/14174518', {'entities': [(84, 100, 'Client')]}), ('"Dear Candidate, Zomato ke sath judiye aur kamayein Rs. 35000 har mahine, saath hi Rs.2000 se 5000 tak ka joining bonus bhi paayein*. Zomato ke saath food delivery karein aur paayein weekly payment + Insurance benefit + Flexible work timing(Full time/Part time available). OnlineÃ‚Â\xa0register karne ke liye neeche diye link se Zomato app download karein aur register karein. Zomato app link:bit.ly/zomatodeliveryapp,Ã‚Â\xa0Joining process ke baare me aur jaankari ke liye yah video dekhein:https://youtu.be/5SpXmdITXDo', {'entities': [(134, 140, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator in SRM Consulting Service at Hassan, Karnataka. Salary 8500 to 15000 + Benefits. Apply here: https://jobalertbot.com/u/22875636/1/14174545', {'entities': [(46, 68, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Mason in Solapur, Maharashtra with EQUICORN MULTITRADE PRIVATE LIMITED under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 14500 to 18000 + Benefits. Apply. https://jobalertbot.com/u/22241406/1/14169072', {'entities': [(94, 129, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21071532/1/14168887', {'entities': [(46, 62, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at KPR Mill in Coimbatore, Tamil Nadu. Salary 10000 to 12000+Benefits. Apply here: https://jobalertbot.com/u/21202726/1/14168929', {'entities': [(46, 54, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Sewing Machine Operator in Bangalore with Silver Spark Apparel Limited under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 12000 to 15000 + Benefits. Apply. https://jobalertbot.com/u/20167123/1/14168805', {'entities': [(101, 129, 'Client')]}), ('Urgent: Hiring for Product stitching Operator in Shree Rajasthan Syntex Limited at Dungarpur, Rajasthan. Salary 12000 to 15000. Apply here: https://jobalertbot.com/u/22996477/1/14174528', {'entities': [(49, 79, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Sewing Machine Operator in Bangalore with Silver Spark Apparel Limited under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 12000 to 15000 + Benefits. Apply. https://jobalertbot.com/u/20166653/1/14168805', {'entities': [(101, 129, 'Client')]}), ('Urgent: Hiring for Housekeeping Executive at RVJ Facilities Services India Pvt. Ltd. in Noida, Uttar Pradesh. Salary 8500 to 9000+Benefits. Apply here: https://jobalertbot.com/u/21502241/1/14168991', {'entities': [(45, 84, 'Client')]}), ('Urgent: Hiring for Security Guard at Walsons Services Pvt Ltd in Gurgaon, Haryana. Salary 10000 to 35000+ Benefits. Apply here: https://jobalertbot.com/u/21423752/1/14168974', {'entities': [(37, 61, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator(Tailoring) at Stellar Fashion Incorp in Karur, Tamil Nadu. Salary 8000 to 10000. Apply here: https://jobalertbot.com/u/22136722/1/14169074', {'entities': [(57, 79, 'Client')]}), ('Urgent: Hiring for Footwear Maker at Gunjan Enterprises in Agra, Uttar Pradesh. Salary 10500 to 12500. Apply here: https://jobalertbot.com/u/21471094/1/14168983', {'entities': [(37, 55, 'Client')]}), ('Urgent: Hiring for Field Sales Executive(QR,EDC) in Hullect Pvt Ltd (hiring for paytm) at Telangana. Salary 12500 to 19000 +Benefits. Apply here: https://jobalertbot.com/u/22878805/1/14174565', {'entities': [(52, 86, 'Client')]}), ('Urgent: Hiring for General Duty Assistant in Portea Medical at Coimbatore, Tamil Nadu. Salary 10000 to 11000. Apply here: https://jobalertbot.com/u/22706092/1/14174432', {'entities': [(45, 59, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator(Tailoring) at Stellar Fashion Incorp in Karur, Tamil Nadu. Salary 8000 to 10000. Apply here: https://jobalertbot.com/u/22134572/1/14169074', {'entities': [(57, 79, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at SCM Garments in Tiruppur, Tamil Nadu. Salary 9000 to 16000. Apply here: https://jobalertbot.com/u/22667984/1/14174382', {'entities': [(46, 58, 'Client')]}), ('Urgent: Hiring for Domestic Worker at React Homecare Services in Bhubaneshwar, Odisha. Salary 6000 to 10000 + Benefits. Apply here: https://jobalertbot.com/u/22591468/1/14173388', {'entities': [(38, 61, 'Client')]}), ('Urgent: Hiring for Delivery Boys in Techatwork Solutions Pvt Ltd at Gurgaon/Faridabad, Haryana. Salary 14000 to 23000+Benefits. Apply here: https://jobalertbot.com/u/22868357/1/14169292', {'entities': [(36, 64, 'Client')]}), ('Urgent: Hiring for Field Sales Executive(EDC) in Hullect Pvt Ltd(hiring for paytm) at Gujarat. Salary 12500 to 19000 +Benefits. Apply here: https://jobalertbot.com/u/22952321/1/14174572', {'entities': [(49, 82, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Exim Knits Pvt. Ltd. in Tiruppur, Tamil Nadu. Salary 9000 to 15000. Apply here: https://jobalertbot.com/u/23067398/1/14174561', {'entities': [(46, 66, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21073717/1/14168887', {'entities': [(46, 62, 'Client')]}), ('"Dear Candidate, Zomato ke sath judiye aur kamayein Rs. 35000 har mahine, saath hi Rs.2000 se 5000 tak ka joining bonus bhi paayein*. Zomato ke saath food delivery karein aur paayein weekly payment + Insurance benefit + Flexible work timing(Full time/Part time available). OnlineÃ‚Â\xa0register karne ke liye neeche diye link se Zomato app download karein aur register karein. Zomato app link:bit.ly/zomatodeliveryapp,Joining process ke baare me aur jaankari ke liye yah video dekhein:https://youtu.be/5SpXmdITXDo', {'entities': [(17, 23, 'Client')]}), ('Urgent: Hiring for Footwear Maker at Gunjan Enterprises in Agra, Uttar Pradesh. Salary 10500 to 12500. Apply here: https://jobalertbot.com/u/21469706/1/14168983', {'entities': [(37, 55, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Sewing Machine Operator in Tiruppur, Tamil Nadu with KPR Mill under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 10000 to 12000 + Benefits. Apply. https://jobalertbot.com/u/21204865/1/14168934', {'entities': [(112, 120, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Courier Warehouse Worker in Bilaspur, Jaipur, with SLV Manpower Solutions under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 7000 to 10000 + Benefits. Apply. https://jobalertbot.com/u/22928638/1/14174558', {'entities': [(110, 132, 'Client')]}), ('BANDHAN BANK IS HIRING !! Interview on 25th Jan (9am to 2pm). Click the link below to register https://forms.gle/PwCHgCwvP9j3QwL86', {'entities': [(0, 12, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Delivery Boys in Bangalore withÃ‚Â\xa0 Prince HR services private limited under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 25000 to 35000 + Benefits. Apply. https://jobalertbot.com/u/22177638/1/14169070', {'entities': [(93, 127, 'Client')]}), ('Urban Clap is Hiring Beauticians in Pune. Earn up to 45000/- per month+ Insurance benefits. Click the link below to apply https://jobalertbot.com/u/17269723/1/14167442', {'entities': [(0, 10, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Anugraha Fashion Mill Pvt Ltd in Korba, Chhatisgarh. Salary 7500 to 12000. Apply here: https://jobalertbot.com/u/22771650/1/14174439', {'entities': [(46, 75, 'Client')]}), ('Urgent: Hiring for Electrician in Bhartendra Daksh Worldwide Solutions Pvt Ltd at Gurgaon, Haryana. Salary 11000 to 11300. Apply here: https://jobalertbot.com/u/23021631/1/14174552', {'entities': [(34, 78, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Shakthi Knitting Ltd in Tiruppur, Tamil Nadu. Salary 9000 to 15000. Apply here: https://jobalertbot.com/u/23005983/1/14174522', {'entities': [(46, 66, 'Client')]}), ('Urgent: Hiring for Sales Representative at Vodafone Idea Ltd in Pune, Maharashtra. Salary 12000 to 15000 + Benefits. Apply here: https://jobalertbot.com/u/21279971/1/14168945', {'entities': [(43, 60, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at KPR Mill in Coimbatore, Tamil Nadu. Salary 10000 to 12000+Benefits. Apply here: https://jobalertbot.com/u/21202683/1/14168929', {'entities': [(46, 54, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Anugraha Fashion Mill Pvt Ltd in Sambalpur, Odisha. Salary 7500 to 12000. Apply here: https://jobalertbot.com/u/22781678/1/14174438', {'entities': [(46, 75, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Shakthi Knitting Ltd in Tiruppur, Tamil Nadu. Salary 9000 to 15000. Apply here: https://jobalertbot.com/u/23008992/1/14174522', {'entities': [(46, 66, 'Client')]}), ('Urgent: Hiring for Garment Cutter at Fortune Inc in Agra, Uttar Pradesh. Salary 9000 to 10500. Apply here: https://jobalertbot.com/u/22988122/1/14174534', {'entities': [(37, 48, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at SCM Garments in Tiruppur, Tamil Nadu. Salary 9000 to 16000. Apply here: https://jobalertbot.com/u/22661141/1/14174382', {'entities': [(46, 58, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Delivery Boys in Bangalore withÃ‚Â\xa0 Prince HR services private limited under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 25000 to 35000 + Benefits. Apply. https://jobalertbot.com/u/22195292/1/14169070', {'entities': [(93, 127, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator in JK International Unit 2 at Tiruppur. Salary 10000 to 12000. Apply here: https://jobalertbot.com/u/22963257/1/14174499', {'entities': [(46, 62, 'Client')]}), ('Urgent: Hiring for Hair Stylist at Vibes in Delhi. Salary 12000 to 18000. Apply here: https://jobalertbot.com/u/22846529/1/14174480', {'entities': [(35, 40, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator(Tailoring) in Deekay Knit Wear at Tiruppur, Tamil Nadu. Salary 10000 to 12000+Benefits.Ã‚Â\xa0Apply here: https://jobalertbot.com/u/21500320/1/14168990', {'entities': [(57, 73, 'Client')]}), ('Urgent: Hiring for Core Shooting Machine Operator at HSIL Limited in Devri, Jhalawar, Rajasthan. Salary 10000 to 12000. Apply here: https://jobalertbot.com/u/21754237/1/14169041', {'entities': [(53, 65, 'Client')]}), ('Urgent: Hiring for Helpers at Innovesource in Hassan, Karnataka. Salary 9000 to 20000+Benefits. Apply here: https://jobalertbot.com/u/21404876/1/14168967', {'entities': [(30, 42, 'Client')]}), ('Urban Clap Hiring AC Technician !! with an Attractive Salary up to 40000/- per month interested? Click the link below https://jobalertbot.com/u/19724561/1/14167457', {'entities': [(0, 10, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Riders in Hyderabad with moEVing Urban Technologies Pvt ltd, under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 12000 to 12500 + Benefits. Apply. https://jobalertbot.com/u/22440265/1/14169449', {'entities': [(84, 118, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Anugraha Fashion Mill Pvt Ltd in Korba, Chhatisgarh. Salary 7500 to 12000. Apply here: https://jobalertbot.com/u/22772876/1/14174439', {'entities': [(46, 75, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Mason in Solapur, Maharashtra with EQUICORN MULTITRADE PRIVATE LIMITED under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 14500 to 18000 + Benefits. Apply. https://jobalertbot.com/u/22237654/1/14169072', {'entities': [(94, 129, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Customer Support Executive,(Voice) in Bangalore, with Sunshine Manpower Solution And Services under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 12000 to 15000 + Benefits. Apply. https://jobalertbot.com/u/23064752/1/14174510', {'entities': [(113, 152, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Sewing Machine Operator in Tiruppur, Tamil Nadu with KPR Mill under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 10000 to 12000 + Benefits. Apply. https://jobalertbot.com/u/21206444/1/14168934', {'entities': [(112, 120, 'Client')]}), ('Urgent: Hiring for Housekeeping Executive at RVJ Facilities Services India Pvt. Ltd. in Noida, Uttar Pradesh. Salary 8500 to 9000+Benefits. Apply here: https://jobalertbot.com/u/21503896/1/14168991', {'entities': [(45, 84, 'Client')]}), ('Urgent: Hiring for Kitchen Helper in Sprint India Services at Mumbai, Maharashtra. Salary 8000 to 10000. Apply here: https://jobalertbot.com/u/22325139/1/14169079', {'entities': [(37, 58, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Tailor / Sewing machine operator in Tiruppur, with REAL TEXTILES under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 8000 to 10000 + Benefits. Apply. https://jobalertbot.com/u/22971405/1/14174560', {'entities': [(110, 123, 'Client')]}), ('Urgent: Hiring for Modular Furniture Assembler in Installco Wify Technology at Pune, Maharashtra. Salary 15000 to 25000+Benefits.Ã‚Â\xa0Ã‚Â\xa0Apply here: https://jobalertbot.com/u/22374337/1/14169395', {'entities': [(50, 75, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Courier Warehouse Worker in Bilaspur, Jaipur, with SLV Manpower Solutions under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 7000 to 10000 + Benefits. Apply. https://jobalertbot.com/u/22927382/1/14174558', {'entities': [(110, 132, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator in Aathava Garments India Pvt Ltd at Coimbatore, Tamil nadu. Salary 9000 to 14000. Apply here: https://jobalertbot.com/u/22724616/1/14174441', {'entities': [(46, 76, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Riders in Hyderabad with moEVing Urban Technologies Pvt ltd, under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 12000 to 12500 + Benefits. Apply. https://jobalertbot.com/u/22440632/1/14169449', {'entities': [(84, 118, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Courier Delivery Executive in Neemrana, Rajasthan with PNG HR SERVICE under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 8000 to 8500 + Benefits. Apply. https://jobalertbot.com/u/21014587/1/14168862', {'entities': [(114, 128, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at SCM Garments in Tiruppur, Tamil Nadu. Salary 9000 to 16000. Apply here: https://jobalertbot.com/u/22662324/1/14174382', {'entities': [(46, 58, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Sealink Fashion Pvt Ltd in Tiruppur, Tamil Nadu. Salary 9000 to 12000. Apply here: https://jobalertbot.com/u/22648629/1/14174380', {'entities': [(46, 69, 'Client')]}), ('Urgent: Hiring for Paver Operator in T.K. EngineeringÃ‚Â\xa0 at Papum Pare, Arunachal Pardesh. Salary 10000 to 20000. Apply here: https://jobalertbot.com/u/22309233/1/14169094', {'entities': [(37, 53, 'Client')]}), ('Urgent: Hiring for Driver at Indra Manpower Consultancy in Nellore, Andhra Pradesh. Salary 10000 to 12000. Apply here: https://jobalertbot.com/u/22826143/1/14174490', {'entities': [(29, 55, 'Client')]}), ('Rs 5000 Starting Bonus!!! Join Zomato as a Food Delivery Partner and earn upto Rs 40,000 per month. Join today to get Rs.5000 Starting Bonus. Hurry up!!This is a limited period offer only for you,Ã‚Â\xa0visit Betterplace experience centre, 3rd floor market square mall, Madivala, Hosur road, Bangalore for immediate joining. Address link:https://maps.app.goo.gl/pG6BE1XdGU3Sac7d9', {'entities': [(31, 37, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Shakthi Knitting Ltd in Tiruppur, Tamil Nadu. Salary 9000 to 15000. Apply here: https://jobalertbot.com/u/23008894/1/14174522', {'entities': [(46, 66, 'Client')]}), ('Urgent: Hiring for Relationship Executive at Velate Recruitment in Jaipur, Rajasthan. Salary 10000 to 30000 + Benefits. Apply here: https://jobalertbot.com/u/21183474/1/14168920', {'entities': [(45, 63, 'Client')]}), ('Urgent: Hiring for Sales Representative at Vodafone Idea Ltd in Pune, Maharashtra. Salary 12000 to 15000 + Benefits. Apply here: https://jobalertbot.com/u/21279617/1/14168945', {'entities': [(43, 60, 'Client')]}), ('Urgent: Hiring for General Duty Assistant in Portea Medical at Coimbatore, Tamil Nadu. Salary 10000 to 11000. Apply here: https://jobalertbot.com/u/22707321/1/14174432', {'entities': [(45, 59, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21103062/1/14168887', {'entities': [(46, 62, 'Client')]}), ('Urgent: Hiring for Garment Cutter at Fortune Inc in Agra, Uttar Pradesh. Salary 9000 to 10500. Apply here: https://jobalertbot.com/u/22992613/1/14174534', {'entities': [(37, 48, 'Client')]}), ('Zomato Mega Hiring!! Join as a food delivery partner and start earning upto Rs. 40000 per month also get a joining bonus of Rs. 5000/-. This is limited period offer only for you!! Download the app andÃ‚Â\xa0register for joining today:https://bit.ly/zomatodeliveryapp', {'entities': [(0, 6, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Adarsh Knit Wear Pvt Ltd in Tiruppur, Tamil Nadu. Salary 9000 to 12000. Apply here: https://jobalertbot.com/u/23056769/1/14174550', {'entities': [(46, 70, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21087642/1/14168887', {'entities': [(46, 62, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Riders in Hyderabad with moEVing Urban Technologies Pvt ltd, under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 12000 to 12500 + Benefits. Apply. https://jobalertbot.com/u/22441268/1/14169449', {'entities': [(84, 118, 'Client')]}), ('Urgent: Hiring for Kitchen Helper in Sprint India Services at Mumbai, Maharashtra. Salary 8000 to 10000. Apply here: https://jobalertbot.com/u/22324208/1/14169079', {'entities': [(37, 58, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Courier Delivery Executive in Neemrana, Rajasthan with PNG HR SERVICE under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 8000 to 8500 + Benefits. Apply. https://jobalertbot.com/u/21014000/1/14168862', {'entities': [(114, 128, 'Client')]}), ('Urgent: Hiring for Modular Furniture Assembler in Installco Wify Technology at Pune, Maharashtra. Salary 15000 to 25000+Benefits.Ã‚Â\xa0Ã‚Â\xa0Apply here: https://jobalertbot.com/u/22377367/1/14169395', {'entities': [(50, 75, 'Client')]}), ('Limited Offer! Zomato is hiring in Hyderabad!! Joining bonus of Rs. 3000 also earn upto Rs. 35000 per month. Offer ends soon!* ForÃ‚Â\xa0joining visit address:https://goo.gl/maps/o2JvTEoRXVwDQKhD6', {'entities': [(15, 21, 'Client')]}), ('Rs 10,000 Starting Bonus!!! Join Zomato as a Food Delivery Partner and earn up to Rs 40,000 per month. Join today to get Rs.10,000 Starting Bonus. Hurry up!! Give a missed call to 08061970811 or directly visit Betterplace experience centre, 3rd-floor market square mall, Madivala, Hosur Road, Bangalore. Address link:https://maps.app.goo.gl/pG6BE1XdGU3Sac7d9Ã‚Â\xa0for direct joining.', {'entities': [(33, 39, 'Client')]}), ('Urgent: Hiring for Mitr(Promoter) at Manpower Group in Telangana. Salary 10200 to 13500+Benefits. Apply here: https://jobalertbot.com/u/21416249/1/14168969', {'entities': [(19, 33, 'Client')]}), ('Urgent: Hiring for Welding Machine Setter/Master Welder in Vone India Services Pvt Ltd at Ganjam/Cuttack, Odisha. Salary 10000 to 12000. Apply here: https://jobalertbot.com/u/22917272/1/14174583', {'entities': [(59, 86, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21087434/1/14168887', {'entities': [(46, 62, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator(Tailoring) at Stellar Fashion Incorp in Karur, Tamil Nadu. Salary 8000 to 10000. Apply here: https://jobalertbot.com/u/22131309/1/14169074', {'entities': [(57, 79, 'Client')]}), ('Urgent: Hiring for Helpers at Innovesource in Hassan, Karnataka. Salary 9000 to 20000+Benefits. Apply here: https://jobalertbot.com/u/21399751/1/14168967', {'entities': [(30, 42, 'Client')]}), ('Urgent: Hiring for Electrician in Bhartendra Daksh Worldwide Solutions Pvt Ltd at Gurgaon, Haryana. Salary 11000 to 11300. Apply here: https://jobalertbot.com/u/23019685/1/14174552', {'entities': [(34, 78, 'Client')]}), ('Urgent: Hiring for Driver at Indra Manpower Consultancy in Nellore, Andhra Pradesh. Salary 10000 to 12000. Apply here: https://jobalertbot.com/u/22829684/1/14174490', {'entities': [(29, 55, 'Client')]}), ('Dear Candidate, greetings from Zomato!!Join Zomato as a food delivery partner and earn upto Rs. 8000 weekly or Rs 35000 per month.Ã‚Â\xa0Joining bonus of Rs.2500/- t&c apply.Ã‚Â\xa0This is a limited period offer only for you!!Hurry up and download the app(Link:https://bit.ly/zomatodeliveryapp) and join today. Thank you!!', {'entities': [(44, 50, 'Client')]}), ('Urgent: Hiring for General Duty Assistant in Portea Medical at Coimbatore, Tamil Nadu. Salary 10000 to 11000. Apply here: https://jobalertbot.com/u/22698289/1/14174432', {'entities': [(45, 59, 'Client')]}), ('Urgent: Hiring for Cook at Housmaid in Gurgaon, Haryana. Salary 10000 to 20000+ Benefits. Apply here: https://jobalertbot.com/u/21425346/1/14168975', {'entities': [(27, 35, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Delivery Boys in Bangalore withÃ‚Â\xa0 Prince HR services private limited under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 25000 to 35000 + Benefits. Apply. https://jobalertbot.com/u/22171295/1/14169070', {'entities': [(93, 127, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Anugraha Fashion Mill Pvt Ltd in Sambalpur, Odisha. Salary 7500 to 12000. Apply here: https://jobalertbot.com/u/22789634/1/14174438', {'entities': [(46, 75, 'Client')]}), ('Urgent: Hiring for ITI Carpenter in Raavela Interiors Pvt. Ltd. at Hyderabad, Telangana. Salary 8000 to 12000. Apply here: https://jobalertbot.com/u/22384945/1/14169270', {'entities': [(36, 63, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at SCM Garments in Tiruppur, Tamil Nadu. Salary 9000 to 16000. Apply here: https://jobalertbot.com/u/22669000/1/14174382', {'entities': [(46, 58, 'Client')]}), ('Urgent: Hiring for Kitchen Helper in Sprint India Services at Mumbai, Maharashtra. Salary 8000 to 10000. Apply here: https://jobalertbot.com/u/22324521/1/14169079', {'entities': [(37, 58, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator in SRM Consulting Service at Hassan, Karnataka. Salary 8500 to 15000 + Benefits. Apply here: https://jobalertbot.com/u/22876359/1/14174545', {'entities': [(46, 68, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at KPR Mill in Tiruppur, Tamil Nadu. Salary 10000 to 12000+Benefits. Apply here: https://jobalertbot.com/u/21203746/1/14168934', {'entities': [(46, 54, 'Client')]}), ('Urgent: Hiring for Housekeeping Executive at RVJ Facilities Services India Pvt. Ltd. in Noida, Uttar Pradesh. Salary 8500 to 9000+Benefits. Apply here: https://jobalertbot.com/u/21508510/1/14168991', {'entities': [(45, 84, 'Client')]}), ('Urgent: Hiring for Desktop Engineer/Operation Support Engineer at Denovo Hitech Solutions Pvt Ltd in Mumbai, Maharashtra. Salary 10000 to 15000. Apply here: https://jobalertbot.com/u/22128369/1/14169073', {'entities': [(66, 97, 'Client')]}), ('Urgent: Hiring for Sewing Machine OperatorÃ‚Â\xa0at Ascent Academy of Training and Development Pvt Ltd. in Tiruppur, Tamil Nadu. Salary 6500 to 8000. Apply here: https://jobalertbot.com/u/22805440/1/14174498', {'entities': [(47, 98, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Shakthi Knitting Ltd in Tiruppur, Tamil Nadu. Salary 9000 to 15000. Apply here: https://jobalertbot.com/u/23008755/1/14174522', {'entities': [(46, 66, 'Client')]}), ('Urgent: Hiring for Field Sales Executive(QR) in Hullect Pvt Ltd(hiring for paytm) at Bangalore. Salary 12500 to 19000 +Benefits. Apply here: https://jobalertbot.com/u/22939536/1/14174571', {'entities': [(48, 81, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Riders in Pune with moEVing Urban Technologies Pvt ltd, under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 12000 to 12500 + Benefits. Apply. https://jobalertbot.com/u/22429289/1/14169447', {'entities': [(79, 113, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator in Aathava Garments India Pvt Ltd at Coimbatore, Tamil nadu. Salary 9000 to 14000. Apply here: https://jobalertbot.com/u/22725430/1/14174441', {'entities': [(46, 76, 'Client')]}), ('Urgent: Hiring for Castering at HSIL Limited in Devri, Jhalawar, Rajasthan. Salary 10000 to 13000. Apply here: https://jobalertbot.com/u/21753670/1/14169039', {'entities': [(32, 44, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Sales Officer/Loan Officer in Gwalior, Madhya Pradesh with HDB Financial Services Ltd under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 9000 to 14000 + Benefits. Apply. https://jobalertbot.com/u/21600924/1/14169023', {'entities': [(118, 144, 'Client')]}), ('Urgent: Hiring for Area Sales Manager at Velate Recruitment in Jaipur, Rajasthan. Salary 20000 to 30000+Benefits. Apply here: https://jobalertbot.com/u/21770571/1/14169031', {'entities': [(41, 59, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21090020/1/14168887', {'entities': [(46, 62, 'Client')]}), ('Urgent: Hiring for Footwear Maker at Gunjan Enterprises in Agra, Uttar Pradesh. Salary 10500 to 12500. Apply here: https://jobalertbot.com/u/21471131/1/14168983', {'entities': [(37, 55, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator in SRM Consulting Service at Hassan, Karnataka. Salary 8500 to 15000 + Benefits. Apply here: https://jobalertbot.com/u/22875175/1/14174545', {'entities': [(46, 68, 'Client')]}), ('Urgent: Hiring for Garment Cutter at Fortune Inc in Agra, Uttar Pradesh. Salary 9000 to 10500. Apply here: https://jobalertbot.com/u/22992610/1/14174534', {'entities': [(37, 48, 'Client')]}), ('Urgent: Hiring for Packing at Auro Weaving Mills(Vardhaman Textile Ltd) in Baddi, Himachal Pradesh. Salary 9200 to 12000+Benefits. Apply here: https://jobalertbot.com/u/21742909/1/14169006', {'entities': [(30, 71, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator(Tailoring) at Deekay Knit Wear in Tiruppur, Tamil Nadu. Salary 10000 to 12000+Benefits.Ã‚Â\xa0Apply here: https://jobalertbot.com/u/21499649/1/14168990', {'entities': [(57, 73, 'Client')]}), ('Urgent: Hiring for Sales Representative at Vodafone Idea Ltd in Lucknow, Uttar Pradesh. Salary 12000 to 15000+Benefits. Apply here: https://jobalertbot.com/u/22087293/1/14168951', {'entities': [(43, 60, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Sewing Machine Operator in Bangalore with Silver Spark Apparel Limited under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 12000 to 15000 + Benefits. Apply. https://jobalertbot.com/u/20155142/1/14168805', {'entities': [(101, 129, 'Client')]}), ('Urgent: Hiring for Desktop Engineer/Operation Support Engineer at Denovo Hitech Solutions Pvt Ltd in Mumbai, Maharashtra. Salary 10000 to 15000. Apply here: https://jobalertbot.com/u/22126212/1/14169073', {'entities': [(66, 97, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Riders in Hyderabad with moEVing Urban Technologies Pvt ltd, under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 12000 to 12500 + Benefits. Apply. https://jobalertbot.com/u/22441588/1/14169449', {'entities': [(84, 118, 'Client')]}), ('"Dear Candidate, Zomato ki taraf se Namaskar!! Zomato ke sath judiye aur kamayein Rs. 35000 har mahine ya Rs. 8000 har hafte + Rs.Ã‚Â\xa02000 se 5000 tak ka joining bonus paayein*. Zomato ke saath food delivery karein aur paayein weekly payment + Insurance benefit + Flexible work timing(Full time/Part time available).Joining process ke baare me aur jaankari ke liye yah video dekhein:https://youtu.be/5SpXmdITXDoÃ‚Â\xa0Online register karne ke liye neeche diye link se app download karein aur register karein. Zomato app link:bit.ly/zomatodeliveryapp', {'entities': [(17, 23, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Courier Delivery Executive in Bhiwadi, Rajasthan with PNG HR SERVICE under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 8000 to 8500 + Benefits. Apply. https://jobalertbot.com/u/21007959/1/14168864', {'entities': [(113, 127, 'Client')]}), ('Urgent: Hiring for Floor Staff at Omisha Solutions in Jaipur, Rajasthan. Salary 12000 to 15000. Apply here: https://jobalertbot.com/u/22295322/1/14169065', {'entities': [(34, 50, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21082894/1/14168887', {'entities': [(46, 62, 'Client')]}), ('Zomato hiring in Bangalore!! Joining bonus Rs. 5000/- Earnings upto Rs. 40000 per month. Offer ends soon!! Watch the video to understand the joining process:https://youtu.be/5SpXmdITXDo, Download the app using the below link from your browser and register online for joining. Zomato app link:bit.ly/zomatodeliveryapp', {'entities': [(0, 6, 'Client')]}), ('Urgent: Hiring for Field Sales Executive at Bridging Gaps in Central Delhi, Delhi. Salary 12000 to 20000. Apply here: https://jobalertbot.com/u/23010531/1/14174543', {'entities': [(44, 57, 'Client')]}), ('Urgent: Hiring for Telecallers at UU Cash Technologies Pvt Ltd in Bangalore, Karnataka. Salary 10000 to 15000. Apply here: https://jobalertbot.com/u/22775791/1/14174489', {'entities': [(34, 62, 'Client')]}), ('Urgent: Hiring for Maid/Babysitter at Housmaid in Gurgaon, Haryana. Salary 10000 to 20000+ Benefits. Apply here: https://jobalertbot.com/u/21425944/1/14168976', {'entities': [(38, 46, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at CMV Global Clothing in Tiruppur, Tamil Nadu. Salary 9500 to 15000. Apply here: https://jobalertbot.com/u/22653558/1/14174381', {'entities': [(46, 65, 'Client')]}), ('Urgent: Hiring for Worker- Floor Staff at Navagiri Apparel in Tiruppur, Tamil Nadu. Salary 7000 to 9000. Apply here: https://jobalertbot.com/u/22388946/1/14169282', {'entities': [(42, 58, 'Client')]}), ('"Dear Candidate, Zomato ke sath judiye aur kamayein Rs. 35000 har mahine, saath hi Rs.2000 se 5000 tak ka joining bonus bhi paayein*. Zomato ke saath food delivery karein aur paayein weekly payment + Insurance benefit + Flexible work timing(Full time/Part time available). OnlineÃ‚Â\xa0register karne ke liye neeche diye link se Zomato app download karein aur register karein. Zomato app link:bit.ly/zomatodeliveryapp,Joining process ke baare me aur jaankari ke liye yah video dekhein:https://youtu.be/5SpXmdITXDo', {'entities': [(17, 23, 'Client')]}), ('Urgent: Hiring for ITI Carpenter in Raavela Interiors Pvt. Ltd. at Hyderabad, Telangana. Salary 8000 to 12000. Apply here: https://jobalertbot.com/u/22385221/1/14169270', {'entities': [(36, 63, 'Client')]}), ('Rs 2400 Starting Bonus!!! Join Zomato as a Food Delivery Partner in Hyderabad and earn up to Rs 40,000 per month+Incentives + other benefits. Join today to get Rs 2400 Starting Bonus. Hurry up!!! Give a missed call to 08061930053 or directly visit address:https://goo.gl/maps/o2JvTEoRXVwDQKhD6 for direct joining.', {'entities': [(31, 37, 'Client')]}), ('"Dear Candidate, Zomato ki taraf se Namaskar!! Zomato ke sath judiye aur kamayein Rs. 35000 har mahine ya Rs. 8000 har hafte saath hi Rs.2000 se 5000 tak ka joining bonus bhi paayein*. Zomato ke saath food delivery karein aur paayein weekly payment + Insurance benefit + Flexible work timing(Full time/Part time available). OnlineÃ‚Â\xa0register karne ke liye neeche diye link ko copy karein aur Zomato app download kar register karein. Zomato app link:bit.ly/zomatodeliveryapp,Ã‚Â\xa0Joining process ke baare me aur jaankari ke liye yah video dekhein:https://youtu.be/5SpXmdITXDo', {'entities': [(17, 23, 'Client')]})]

In [10]:
VALD1 = [('URBANCLAP HIRING! Ac Technician - Male. Earn up to 40,000/- per month. Kindly click the linkÃ‚Â\xa0https://forms.gle/Tx5br3AbACU7KGDv8Ã‚Â\xa0', {'entities': [(0, 9, 'Client')]}), ('Urgent: Hiring for Marketing Executive at Indra Manpower Consultancy in Nellore, Andhra Pradesh. Salary 10000 to 15000. Apply here: https://jobalertbot.com/u/22802164/1/14174479', {'entities': [(42, 68, 'Client')]}), ('Urgent: Hiring for Sales Representative at Master Jobs in Hyderabad, Telangana. Salary 15000 to 18000. Apply here: https://jobalertbot.com/u/22194484/1/14169076', {'entities': [(43, 54, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Courier Warehouse Worker in Bilaspur, Jaipur, with SLV Manpower Solutions under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 7000 to 10000 + Benefits. Apply. https://jobalertbot.com/u/22928749/1/14174558', {'entities': [(110, 132, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21088812/1/14168887', {'entities': [(46, 62, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at SCM Garments in Tiruppur, Tamil Nadu. Salary 9000 to 16000. Apply here: https://jobalertbot.com/u/22668441/1/14174382', {'entities': [(46, 58, 'Client')]}), ('Urgent: Hiring for Customer Care Executive in Avant Career at Delhi. Salary 9000 to 12000. Apply here: https://jobalertbot.com/u/20246951/1/14168762', {'entities': [(46, 58, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Anugraha Fashion Mill Pvt Ltd in Sambalpur, Odisha. Salary 7500 to 12000. Apply here: https://jobalertbot.com/u/22781778/1/14174438', {'entities': [(46, 75, 'Client')]}), ('Urgent: Hiring for Delivery Boys in Techatwork Solutions Pvt Ltd at Gurgaon/Faridabad, Haryana. Salary 14000 to 23000+Benefits. Apply here: https://jobalertbot.com/u/22869771/1/14169292', {'entities': [(36, 64, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at SCM Garments in Tiruppur, Tamil Nadu. Salary 9000 to 16000. Apply here: https://jobalertbot.com/u/22660259/1/14174382', {'entities': [(46, 58, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Sealink Fashion Pvt Ltd in Tiruppur, Tamil Nadu. Salary 9000 to 12000. Apply here: https://jobalertbot.com/u/22650317/1/14174380', {'entities': [(46, 69, 'Client')]}), ('Urgent: Hiring for Kitchen Helper in Sprint India Services at Mumbai, Maharashtra. Salary 8000 to 10000. Apply here: https://jobalertbot.com/u/22324548/1/14169079', {'entities': [(37, 58, 'Client')]}), ('"Dear Candidate, Zomato ki taraf se Namaskar!! Zomato ke sath judiye aur kamayein Rs. 35000 har mahine ya Rs. 8000 har hafte saath hi Rs.2000 se 5000 tak ka joining bonus bhi paayein*. Zomato ke saath food delivery karein aur paayein weekly payment + Insurance benefit + Flexible work timing(Full time/Part time available). OnlineÃ‚Â\xa0register karne ke liye neeche diye link ko copy karein aur Zomato app download kar register karein. Zomato app link:bit.ly/zomatodeliveryapp,Ã‚Â\xa0Joining process ke baare me aur jaankari ke liye yah video dekhein:https://youtu.be/5SpXmdITXDo', {'entities': [(17, 23, 'Client')]}), ("India's No.1 Bank is hiring Relationship Executives via Randstad, Earn up to 18000 /- plus incentives per month. Only 100 Open positions, Click below to schedule your Interview. https://jobalertbot.com/u/20993459/1/14168873", {'entities': [(56, 64, 'Client')]}), ('Urgent: Hiring for Dental Assistant at SRM Consulting Service in Kurnool, Andhra Pradesh. Salary 20000 to 50000. Apply here: https://jobalertbot.com/u/22392284/1/14169309', {'entities': [(39, 61, 'Client')]}), ('Urgent: Hiring for Patient Care at React Homecare Services in Cuttack/Bhubaneshwar/Rourkela, Odisha. Salary 7000 to 12000 + Benefits. Apply here: https://jobalertbot.com/u/22589862/1/14173387', {'entities': [(35, 58, 'Client')]}), ('Rs 2400 Starting Bonus!!! Join Zomato as a Food Delivery Partner in Hyderabad and earn upto Rs 40,000 per month+Incentives + other benefits. Join today to get Rs 2400 Starting Bonus. Hurry up!!! Give a missed call to 08061930053 or directly visit address:https://goo.gl/maps/o2JvTEoRXVwDQKhD6 for direct joining.', {'entities': [(31, 37, 'Client')]}), ('Urgent: Hiring for Kitchen Helper in Sprint India Services at Mumbai, Maharashtra. Salary 8000 to 10000. Apply here: https://jobalertbot.com/u/22326514/1/14169079', {'entities': [(37, 58, 'Client')]}), ('Urgent: Hiring for Sewing Machine OperatorÃ‚Â\xa0at Ascent Academy of Training and Development Pvt Ltd. in Tiruppur, Tamil Nadu. Salary 6500 to 8000. Apply here: https://jobalertbot.com/u/22805437/1/14174498', {'entities': [(47, 98, 'Client')]}), ('Urgent: Hiring for Footwear Maker at Gunjan Enterprises in Agra, Uttar Pradesh. Salary 10500 to 12500. Apply here: https://jobalertbot.com/u/21476442/1/14168983', {'entities': [(37, 55, 'Client')]}), ('Urgent: Hiring for Mitr(Promoter) at Manpower Group in Telangana. Salary 10200 to 13500+Benefits. Apply here: https://jobalertbot.com/u/21417581/1/14168969', {'entities': [(19, 33, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21076196/1/14168887', {'entities': [(46, 62, 'Client')]}), ('Urgent: Hiring for Housekeeping Executive at RVJ Facilities Services India Pvt. Ltd. in Noida, Uttar Pradesh. Salary 8500 to 9000+Benefits. Apply here: https://jobalertbot.com/u/21505758/1/14168991', {'entities': [(45, 84, 'Client')]}), ('This is an interview reminder message. Your interview is scheduled with Dar Human Resources today at N/A location. Please reach location on time and carry required documents. Only few slots left! Interviewer Name : Farooq Shaikh Interviewer Contact Number : +919673191193', {'entities': [(72, 91, 'Client')]}), ('Urgent: Hiring for Finishing Goods & Garment at Fortune Inc in Agra, Uttar Pradesh. Salary 9000 to 10500. Apply here: https://jobalertbot.com/u/22991950/1/14174535', {'entities': [(48, 59, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator(Tailoring) in Deekay Knit Wear at Tiruppur, Tamil Nadu. Salary 10000 to 12000+Benefits.Ã‚Â\xa0Apply here: https://jobalertbot.com/u/21500373/1/14168990', {'entities': [(57, 73, 'Client')]}), ('Urgent: Hiring for Telecaller at Somsan Advisory Services LLP in Gautam Buddh Nagar, Noida. Salary 8000 to 22000. Apply here:Ã‚Â\xa0 https://jobalertbot.com/u/21247475/1/14168926', {'entities': [(33, 61, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Sewing Machine Operator in Bangalore with Silver Spark Apparel Limited under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 12000 to 15000 + Benefits. Apply. https://jobalertbot.com/u/20153426/1/14168805', {'entities': [(101, 129, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at CMV Global Clothing in Tiruppur, Tamil Nadu. Salary 9500 to 15000. Apply here: https://jobalertbot.com/u/22654450/1/14174381', {'entities': [(46, 65, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Sewing Machine Operator in Coimbatore, Tamil Nadu with KPR Mill under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 10000 to 12000 + Benefits. Apply. https://jobalertbot.com/u/21203108/1/14168929', {'entities': [(114, 122, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Tailor / Sewing machine operator in Tiruppur, with REAL TEXTILES under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 8000 to 10000 + Benefits. Apply. https://jobalertbot.com/u/22973540/1/14174560', {'entities': [(110, 123, 'Client')]}), ('Urgent: Hiring for Receptionist at Career Hunt Manpower Solution in Kolkata, West Bengal. Salary 10000 to 12000 + Benefits. Apply here: https://jobalertbot.com/u/22083689/1/14169061', {'entities': [(35, 64, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Sealink Fashion Pvt Ltd in Tiruppur, Tamil Nadu. Salary 9000 to 12000. Apply here: https://jobalertbot.com/u/22648079/1/14174380', {'entities': [(46, 69, 'Client')]}), ('Rs 2400 Starting Bonus!!! Join Zomato as a Food Delivery Partner in Hyderabad and earn upto Rs 40,000 per month+Incentives + other benefits. Join today to get Rs 2400 Starting Bonus. Hurry up!!! Give a missed call to 08061930053 or for direct joining directly visit address:https://goo.gl/maps/o2JvTEoRXVwDQKhD6', {'entities': [(31, 37, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator(Tailoring) at Stellar Fashion Incorp in Karur, Tamil Nadu. Salary 8000 to 10000. Apply here: https://jobalertbot.com/u/22136764/1/14169074', {'entities': [(57, 79, 'Client')]}), ('Urgent: Hiring for Field Sales Executive(QR,EDC) in Hullect Pvt Ltd (hiring for paytm) at Pune. Salary 12500 to 19000 +Benefits. Apply here: https://jobalertbot.com/u/22945476/1/14174568', {'entities': [(52, 86, 'Client')]}), ('Big Basket Mega Hiring! Big Basket Hiring for Part-time Delivery Partners in Bangalore. Earn upto 23000/- + incentives per month. Click on the link below to apply . https://jobalertbot.com/u/21446003/1/14168184', {'entities': [(0, 10, 'Client')]}), ('Urgent: Hiring for Modular Furniture Assembler in Installco Wify Technology at Pune, Maharashtra. Salary 15000 to 25000+Benefits.Ã‚Â\xa0Ã‚Â\xa0Apply here: https://jobalertbot.com/u/22377465/1/14169395', {'entities': [(50, 75, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Roadmine Apparel in Tiruppur, Tamil Nadu. Salary 8000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/21067345/1/14168887', {'entities': [(46, 62, 'Client')]}), ('Urgent: Hiring for Restaurant Manager in Sprint India Services at Mumbai, Maharashtra. Salary 12000 to 15000. Apply here: https://jobalertbot.com/u/22309829/1/14169083', {'entities': [(41, 62, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Riders in Hyderabad with moEVing Urban Technologies Pvt ltd, under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 12000 to 12500 + Benefits. Apply. https://jobalertbot.com/u/22441590/1/14169449', {'entities': [(84, 118, 'Client')]}), ('Urgent: Hiring for Worker- Floor Staff at Navagiri Apparel in Tiruppur, Tamil Nadu. Salary 7000 to 9000. Apply here: https://jobalertbot.com/u/22395867/1/14169282', {'entities': [(42, 58, 'Client')]}), ('Urgent: Hiring for General Duty Assistant in Portea Medical at Coimbatore, Tamil Nadu. Salary 10000 to 11000. Apply here: https://jobalertbot.com/u/22708439/1/14174432', {'entities': [(45, 59, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for theÃ‚Â\xa0 Machine Operators (Automobiles) in Noida with Swapnrachayita Edutech Private Limited under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 8000 to 12000 + Benefits. Apply. https://jobalertbot.com/u/22323195/1/14169088', {'entities': [(107, 145, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Anugraha Fashion Mill Pvt Ltd in Sambalpur, Odisha. Salary 7500 to 12000. Apply here: https://jobalertbot.com/u/22780570/1/14174438', {'entities': [(46, 75, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for theÃ‚Â\xa0 Machine Operators (Automobiles) in Noida with Swapnrachayita Edutech Private Limited under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 8000 to 12000 + Benefits. Apply. https://jobalertbot.com/u/22322470/1/14169088', {'entities': [(107, 145, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at CMV Global Clothing in Tiruppur, Tamil Nadu. Salary 9500 to 15000. Apply here: https://jobalertbot.com/u/22654773/1/14174381', {'entities': [(46, 65, 'Client')]}), ('Urgent: Hiring for Sales Executive at Sivashakthi Group of Companies in Bhopal, Madhya Pradesh. Salary 8500 to 15000+Benefits. Apply here: https://jobalertbot.com/u/20295359/1/14168777', {'entities': [(38, 68, 'Client')]}), ('Urgent: Hiring for Leather Goods & Garments at Fortune Inc in Agra, Uttar Pradesh. Salary 9000 to 10500. Apply here: https://jobalertbot.com/u/22988213/1/14174505', {'entities': [(47, 58, 'Client')]}), ('Dear candidate! Now join Zomato as full time/Part time and get Rs. 2000 joining bonus!! Also earn upto Rs. 35000 per month. Offer ends soon, Hurry up!! Watch this video to know more about the joining process(https://youtu.be/5SpXmdITXDo)Ã‚Â\xa0Download the Zomato app using the below link in your browser and register for joining. App link:bit.ly/zomatodeliveryapp', {'entities': [(25, 31, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Sealink Fashion Pvt Ltd in Tiruppur, Tamil Nadu. Salary 9000 to 12000. Apply here: https://jobalertbot.com/u/22648485/1/14174380', {'entities': [(46, 69, 'Client')]}), ('Urgent: Hiring for Telecaller at Rits Enterprises in Rohini, Moti Nagar- Delhi. Salary 9000 to 15000 + Benefits. Apply here: https://jobalertbot.com/u/22114452/1/14169068', {'entities': [(33, 49, 'Client')]}), ('Rs 2400 Starting Bonus!!! Join Zomato as a Food Delivery Partner in Hyderabad and earn up to Rs 40,000 per month+Incentives + other benefits. Join today to get Rs 2400 Starting Bonus. Hurry up!!! Give a missed call to 08061930053 or directly visit address:https://goo.gl/maps/o2JvTEoRXVwDQKhD6 for direct joining.', {'entities': [(31, 37, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator in Aathava Garments India Pvt Ltd at Coimbatore, Tamil nadu. Salary 9000 to 14000. Apply here: https://jobalertbot.com/u/22724731/1/14174441', {'entities': [(46, 76, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Adarsh Knit Wear Pvt Ltd in Tiruppur, Tamil Nadu. Salary 9000 to 12000. Apply here: https://jobalertbot.com/u/23056640/1/14174550', {'entities': [(46, 70, 'Client')]}), ('Rs 5000 Starting Bonus!!! Join Zomato as a Food Delivery Partner and earn upto Rs 40,000 per month. Join today to get Rs.5000 Starting Bonus. Hurry up!!This is a limited period offer only for you,Ã‚Â\xa0visit Betterplace experience centre, 3rd floor market square mall, Madivala, Hosur road, Bangalore for immediate joining. Address link:https://maps.app.goo.gl/pG6BE1XdGU3Sac7d9', {'entities': [(31, 37, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at CMV Global Clothing in Tiruppur, Tamil Nadu. Salary 9500 to 15000. Apply here: https://jobalertbot.com/u/22655127/1/14174381', {'entities': [(46, 65, 'Client')]}), ('Urgent: Hiring for Marketing Executive at Indra Manpower Consultancy in Nellore, Andhra Pradesh. Salary 10000 to 15000. Apply here: https://jobalertbot.com/u/22797633/1/14174479', {'entities': [(42, 68, 'Client')]}), ('Urgent: Hiring for Assistant Acquisition Manager-Current Accounts/Savings Account in Velate Recruitment at Jaipur, Rajasthan. Salary 20000 to 22000. Apply here: https://jobalertbot.com/u/22462182/1/14169069', {'entities': [(85, 104, 'Client')]}), ('Urgent: Hiring for Telecaller at Somsan Advisory Services LLP in Delhi. Salary 8000 to 22000. Apply here: https://jobalertbot.com/u/21212128/1/14168925', {'entities': [(33, 61, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at CMV Global Clothing in Tiruppur, Tamil Nadu. Salary 9500 to 15000. Apply here: https://jobalertbot.com/u/22655676/1/14174381', {'entities': [(46, 65, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Exim Knits Pvt. Ltd. in Tiruppur, Tamil Nadu. Salary 9000 to 15000. Apply here: https://jobalertbot.com/u/23067005/1/14174561', {'entities': [(46, 66, 'Client')]}), ('Urgent: Hiring for Polisher at Amrapali Exports in Jaipur, Rajasthan. Salary 10000 to 14000 + Benefits. Apply here: https://jobalertbot.com/u/22643077/1/14173401', {'entities': [(31, 47, 'Client')]}), ('Urgent: Hiring for Relationship Executive at Velate Recruitment in Jaipur, Rajasthan. Salary 10000 to 30000 + Benefits. Apply here: https://jobalertbot.com/u/21186187/1/14168920', {'entities': [(45, 63, 'Client')]}), ('Urgent: Hiring for Floor Staff at Omisha Solutions in Gurgaon, Haryana. Salary 12000 to 15000. Apply here: https://jobalertbot.com/u/22294498/1/14169066', {'entities': [(34, 50, 'Client')]}), ('Urgent: Hiring for Worker- Floor Staff at Navagiri Apparel in Tiruppur, Tamil Nadu. Salary 7000 to 9000. Apply here: https://jobalertbot.com/u/22396475/1/14169282', {'entities': [(42, 58, 'Client')]}), ('Urgent: Hiring for Individual Sales Professional/Distributor Salesman/Sales Associate in Inway Retails Pvt Ltd at Delhi. Salary 8000 to 20000. Apply here: https://jobalertbot.com/u/22733207/1/14174434', {'entities': [(89, 110, 'Client')]}), ('Urgent: Hiring for Desktop Engineer/Operation Support Engineer at Denovo Hitech Solutions Pvt Ltd in Mumbai, Maharashtra. Salary 10000 to 15000. Apply here: https://jobalertbot.com/u/22126626/1/14169073', {'entities': [(66, 97, 'Client')]}), ('Urgent: Hiring for Leather Goods & Garments at Fortune Inc in Agra, Uttar Pradesh. Salary 9000 to 10500. Apply here: https://jobalertbot.com/u/22988631/1/14174505', {'entities': [(47, 58, 'Client')]}), ('Urgent: Hiring for Rod Binding/Fitter in EQUICORN MULTIGRADE PRIVATE LIMITED at Solapur, Maharashtra. Salary 10000 to 11000. Apply here: https://jobalertbot.com/u/22345259/1/14169096', {'entities': [(41, 76, 'Client')]}), ('Urgent: Hiring for Worker- Floor Staff at Navagiri Apparel in Tiruppur, Tamil Nadu. Salary 7000 to 9000. Apply here: https://jobalertbot.com/u/22391556/1/14169282', {'entities': [(42, 58, 'Client')]}), ('Urgent: Hiring for Weaving Operator at Auro Weaving Mills(Vardhaman Textile Ltd) in Baddi, Himachal Pradesh. Salary 9200 to 12000+Benefits. Apply here: https://jobalertbot.com/u/21734470/1/14168997', {'entities': [(39, 80, 'Client')]}), ('Urgent: Hiring for Telecaller at Rits Enterprises in Rohini, Moti Nagar- Delhi. Salary 9000 to 15000 + Benefits. Apply here: https://jobalertbot.com/u/22111329/1/14169068', {'entities': [(33, 49, 'Client')]}), ('Urgent: Hiring for Sewing Machine Operator at Sealink Fashion Pvt Ltd in Tiruppur, Tamil Nadu. Salary 9000 to 12000. Apply here: https://jobalertbot.com/u/22648193/1/14174380', {'entities': [(46, 69, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Loading & unloading/helper in Bharuch, Gujarat with Dhriti ventures llp under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 8000 to 12000 + Benefits. Apply. https://jobalertbot.com/u/21057593/1/14168899', {'entities': [(111, 130, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Assembly Line Operator in Bangalore, with ISEE STAFFING SOLUTIONS PVT LTD under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 13000 to 14000 + Benefits. Apply. https://jobalertbot.com/u/23000124/1/14174574', {'entities': [(101, 132, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Plumber for Cooking Gas Pipeline in Mumbai with Dev Loka Educational Trust, under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 11500 to 20000 + Benefits. Apply. https://jobalertbot.com/u/22425213/1/14169444', {'entities': [(107, 133, 'Client')]}), ('Join Zomato as a delivery partner in Bangalore and get Rs 5000 joining bonus also earn upto Rs 40000 per month. Joining address:Betterplace experience centre, 3rd-floor market square mall, Madivala, Hosur Road, Bangalore. Address link:https://maps.app.goo.gl/pG6BE1XdGU3Sac7d9', {'entities': [(5, 11, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the Insurance Agent in Visakhapatnam, with ABBINENI BUSINESS MANAGEMENT PRIVATE LIMITED under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 20000 to 23000 + Benefits. Apply. https://jobalertbot.com/u/23052324/1/14174521', {'entities': [(98, 142, 'Client')]}), ('Dear candidate, Happy New Year!!Mega Hiring in Zomato- Joining Bonus of Rs. 10,000/- Earnings upto 40,000 per month, attracting incentives and much more!!Want to join us as a delivery partner? Please fill out the form and we will reach you!! Form:https://cutt.ly/YjinSglÃ‚Â\xa0Thank you!!!', {'entities': [(47, 53, 'Client')]}), ('Dear candidate, Happy New Year!!Mega Hiring in Zomato- Joining Bonus of Rs. 10,000/- Earnings upto 40,000 per month, attracting incentives and much more!!Want to join us as a delivery partner? Please fill out the form and we will reach you!! Form:https://cutt.ly/YjinSglÃ‚Â\xa0Thank you!!!', {'entities': [(0, 6, 'Client')]}), ('Zomato Mega Hiring! Joining bonus Rs. 5000/- Earnings upto Rs.40000 per month and weekly payments. Offer to end soon. For joiningÃ‚Â\xa0visitÃ‚Â\xa0Betterplace experience centre, 3rd floor market square mall, Madivala, Hosur road, Bangalore. Address link:https://maps.app.goo.gl/pG6BE1XdGU3Sac7d9', {'entities': [(35, 41, 'Client')]}), ('Dear Candidate, You have been shortlisted to apply for the General Duty Assistant (Patient Care) in Ghaziabad with Grandage Services Private Limited (WINAGE) under the Pradhan Mantri Kaushal Vikas Yojana. Salary Rs. 17000 to 23000 + Benefits. Apply. https://jobalertbot.com/u/22302037/1/14169087', {'entities': [(115, 157, 'Client')]}), ('Limited Offer! Zomato is hiring in Bangalore!! Joining bonus of Rs. 5000 also earn upto Rs. 40000 per month. Offer to end soon! For joining visitÃ‚Â\xa0Betterplace experience centre, 3rd floor market square mall, Madivala, Hosur road, Bangalore. Address link:https://maps.app.goo.gl/pG6BE1XdGU3Sac7d9', {'entities': [(15, 21, 'Client')]})]

In [11]:
TRAIN = trim_entity_spans(TRND1)
VAL = trim_entity_spans(VALD1)

#### Step 5: Converting both TRAIN and VAL into .spacy 

In [17]:

import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)


    db.to_disk("./train.spacy") # save the docbin object

100%|█████████████████████████████████████████████████████████████████████████████████| 38/38 [00:00<00:00, 577.29it/s]

Skipping entity
Skipping entity


In [8]:
import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm(VAL): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./val.spacy") # save the docbin object

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 834.36it/s]


#### Step 6: Building and Training the model (Using CLI instead of notebooks is suggested)

*For building the model requires a config.cfg which can be downloaded from spacy v3.0 website. Then it has to be filled as well using fill-config,

python -m spacy init fill-config old_config.cfg config.cfg

##### The below code will build, train and save the model. Also showcases LOSS, SCORE and other parameters on each epoch  (in CLI)
python -m spacy train config.cfg --output ./opf1 --path.train ./train.spacy --path.dev ./val.spacy --gpu-id 0




#### Step 7:  Making the Predictions 

##### Loading our custom model (opf1) and making predictions on few samples

In [9]:
import spacy

nlp = spacy.load(R".\opf1\model-best") #load the best model  #opf1 0.90 score
doc = nlp("Urgent: Hiring for Sewing Machine Operator in Aathava Garments India Pvt Ltd at Coimbatore, Tamil nadu. Salary 9000 to 14000. Apply here: https://jobalertbot.com/u/22724731/1/14174441") # input sample text

et = spacy.displacy.render(doc, style="ent") # display in Jupyter
doc.ents

(Aathava Garments India Pvt Ltd,)

In [10]:
import spacy

nlp = spacy.load(R".\opf1\model-best") #load the best model  #opf1 0.90 score
doc = nlp("Dear Candidate, Zomato ke sath judiye aur kamayein Rs. 35000 har mahine, saath hi Rs.2000 se 5000 tak ka joining bonus bhi paayein*. Zomato ke saath food delivery karein aur paayein weekly payment + Insurance benefit + Flexible work timing(Full time/Part time available). OnlineÃ‚Â\xa0register karne ke liye neeche diye link se Zomato app download karein aur register karein. Zomato app link:bit.ly/zomatodeliveryapp,Ã‚Â\xa0Joining process ke baare me aur jaankari ke liye yah video dekhein:https://youtu.be/5SpXmdITXDo") # input sample text

et = spacy.displacy.render(doc, style="ent") # display in Jupyter
doc.ents

(Zomato,)

In [11]:
import spacy

nlp = spacy.load(R".\opf1\model-best") #load the best model  #opf1 0.90 score
doc = nlp("à¤ªà¥à¤°à¤¿à¤¯ à¤‰à¤®à¥à¤®à¥€à¤¦à¤µà¤¾à¤°! Zomato à¤•à¥€ à¤¤à¤°à¤« à¤¸à¥‡ à¤¨à¤®à¤¸à¥à¤•à¤¾à¤°!! à¤¶à¤¹à¤° à¤®à¥‡à¤‚ Zomato à¤•à¥€ à¤­à¤°à¥à¤¤à¥€ à¤œà¤¾à¤°à¥€ à¤¹à¥ˆ!! à¤•à¤®à¤¾à¤à¤ à¤°à¥à¤ªà¤¯à¥‡ 35000 à¤ªà¥à¤°à¤¤à¤¿ à¤®à¤¾à¤¹ + à¤¸à¤¾à¤ªà¥à¤¤à¤¾à¤¹à¤¿à¤• à¤­à¥à¤—à¤¤à¤¾à¤¨ + à¤¬à¥€à¤®à¤¾ à¤²à¤¾à¤­ à¤‰à¤ªà¤²à¤¬à¥à¤§!! à¤¸à¤¾à¤¥ à¤¹à¥€ 2000-5000 à¤°à¥à¤ªà¤¯à¥‡ à¤•à¤¾ à¤¬à¥‹à¤¨à¤¸ à¤­à¥€ à¤ªà¥à¤°à¤¾à¤ªà¥à¤¤ à¤•à¤°à¥‡à¤‚!! à¤¸à¥€à¤®à¤¿à¤¤ à¤…à¤µà¤§à¤¿ à¤•à¥€ à¤ªà¥‡à¤¶à¤•à¤¶ à¤•à¥‡à¤µà¤² à¤†à¤ªà¤•à¥‡ à¤²à¤¿à¤!!Zomato à¤•à¥‡ à¤¸à¤¾à¤¥ à¤œà¥à¤¡à¤¼à¤¨à¥‡ à¤•à¥‡ à¤²à¤¿à¤ à¤•à¥ƒà¤ªà¤¯à¤¾ à¤…à¤ªà¤¨à¥‡ à¤¬à¥à¤°à¤¾à¤‰à¤œà¤¼à¤° à¤®à¥‡à¤‚ à¤²à¤¿à¤‚à¤• à¤•à¥‹ à¤•à¥‰à¤ªà¥€ à¤•à¤°à¤•à¥‡ Zomato à¤à¤ª à¤¡à¤¾à¤‰à¤¨à¤²à¥‹à¤¡ à¤•à¤°à¥‡à¤‚ à¤”à¤° online à¤°à¤œà¤¿à¤¸à¥à¤Ÿà¤° à¤•à¤°à¥‡à¤‚à¥¤ Zomato à¤à¤ª à¤²à¤¿à¤‚à¤•:https://bit.ly/zomatodeliveryapp, à¤œà¥‰à¤‡à¤¨à¤¿à¤‚à¤— à¤ªà¥à¤°à¤•à¥à¤°à¤¿à¤¯à¤¾ à¤•à¥‡ à¤¬à¤¾à¤°à¥‡ à¤®à¥‡à¤‚ à¤…à¤§à¤¿à¤• à¤œà¤¾à¤¨à¤•à¤¾à¤°à¥€ à¤•à¥‡ à¤²à¤¿à¤, à¤•à¥ƒà¤ªà¤¯à¤¾ à¤¨à¥€à¤šà¥‡ à¤¦à¤¿à¤¯à¤¾ à¤—à¤¯à¤¾ à¤µà¥€à¤¡à¤¿à¤¯à¥‹ à¤¦à¥‡à¤–à¥‡à¤‚:https://youtu.be/5SpXmdITXDo") # input sample text

et = spacy.displacy.render(doc, style="ent") # display in Jupyter
doc.ents

(Zomato, Zomato, Zomato)